In [1]:
import scipy
from scipy.spatial.distance import squareform
import cellgrid
from initialization import init_uniform
import numpy as np
from MDAnalysis.lib.distances import self_distance_array
import itertools

In [2]:
%load_ext memory_profiler
%matplotlib inline

In [12]:
def _check_box(box):
    if box.shape == (6,):
        box1 = box[:3]
    else:
        box1 = box
    return box1

def pair_contact(cg,maxdist):
    indx,dist = cellgrid.cellgrid_self_distance_array(cg)
    mask = np.where(dist<maxdist)
    return indx[mask],dist[mask]

In [4]:
def bf_select(box, points, maxdist):
    distance = self_distance_array(points,box)
    distance = scipy.spatial.distance.squareform(distance)
    distance[np.tril_indices(distance.shape[0])] += (1.1*maxdist)
    mask = np.where((distance < maxdist))
    out = (np.column_stack((mask[0],mask[1])),distance[mask])
    return out

In [5]:
def cg_select(box,cellsize,points,cutoff_distance):
    box = _check_box(box)
    cellsize = max(cellsize,cutoff_distance)
    cg1 = cellgrid.CellGrid(box,cellsize,points)
    out_cg = pair_contact(cg1,cutoff_distance)
    return out_cg

def cg_build(box,cellsize,points,cutoff_distance):
    box = _check_box(box)
    cellsize = max(cellsize,cutoff_distance)
    cg1 = cellgrid.CellGrid(box,cellsize,points)
    return cg1

def cg_select_old(box, points, maxdist):
    box = _check_box(box)
    indx,dist = cellgrid.capped_self_distance_array(points,maxdist,box)
    mask = np.where(dist<maxdist)
    return indx[mask],dist[mask]


# Cellsize > Cutoff_Distance

In [13]:
##Setting
box = np.array([100,100,100,90.,90.,90.],dtype=np.float32)
cellsize = 10.
cutoff_distance = 5.
Npoints = 1000

In [14]:
#Initialization
points = init_uniform(box,Npoints)

In [15]:
out_cg = cg_select(box,cellsize,points,cutoff_distance)

In [16]:
out_bf = bf_select(box,points,cutoff_distance)

In [17]:
out_cgold = cg_select_old(box,points,cutoff_distance)

# Testing

In [18]:
# Length
np.testing.assert_equal(out_bf[1].shape,out_cg[1].shape)
np.testing.assert_equal(out_bf[1].shape,out_cgold[1].shape)
# Distances
np.testing.assert_array_almost_equal(np.sort(out_bf[1].astype(np.float64)),np.sort(out_cg[1]),decimal=5) #Comparing array values
np.testing.assert_array_almost_equal(np.sort(out_bf[1].astype(np.float64)),np.sort(out_cgold[1]),decimal=5)
# Indices
np.testing.assert_array_equal(np.unique(out_cg[0]),np.unique(out_bf[0]))
np.testing.assert_array_equal(np.unique(out_cgold[0]),np.unique(out_bf[0]))

# Variation of cellsize with Npoints and cutoff distance

In [ ]:
#Setting
cellsize = np.linspace(5,25,num=5,dtype=np.int32)
Npoints = np.logspace(2,4.5,num=10,dtype=np.int32)
cutoff = np.linspace(5,25,num=5,dtype=np.float64)
box = np.array([100,100,100,90.,90.,90.],dtype=np.float32)

In [ ]:
cellsize,Npoints,cutoff

In [ ]:
data,old = [],[]
for npt in Npoints:
    points = init_uniform(box,npt)
    print("Npoints : {}".format(npt))
    for cut in cutoff:
        time_build_old = %timeit -q -o -n 10 -r 3 cellgrid.CellGrid(_check_box(box),cut,points)
        time_old = %timeit -q -o -n 10 -r 3 cg_select_old(box,points,cut)
        old.append((npt,cut,time_build_old.average,time_old.average))
        print("Cutoff : {}".format(cut))
        for cs in cellsize:
            time_build = %timeit -q -o -n 10 -r 3 cg_build(_check_box(box),cs,points,cut)
            time = %timeit -q -o -n 10 -r 3 cg_select(box,cs,points,cut)
            print("Cellsize : {}".format(cs))
            #print("Npoints {} CellSize {} Time {}".format(npt,cs,time.average))
            data.append((npt,cs,cut,time.average,time_build.average))

In [ ]:
old,data

# Visualization

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
data

## Fixed Cutoff Distance and variation with Cellsize

In [ ]:
mask = np.where(data[...,2] == 10.)
#print(data[mask])
mask2 = np.where(data[mask][...,1]==5)
print(data[mask][mask2])
plt.plot(Npoints,data[mask][mask2][...,3])

In [ ]:
old = np.array(old)

In [ ]:
old,data

In [ ]:
mask = np.where(old[:,1]==10)
old[mask][:,3]

In [ ]:
c = 20
cut = 25 
mask = np.where(data[:,2]==cut)
mask_cs = np.where(data[mask][:,1]==c)
time = data[mask][mask_cs][:,3]
print(time)

In [ ]:
data,old = np.array(data),np.array(old)
col,row = 2,len(cutoff)//2
fig, (ax) = plt.subplots(row, col, sharex='col', sharey='row')
fig.subplots_adjust(hspace=0.7, wspace=0.5)
fig.set_size_inches(15,15)
#ax = np.array([ax])
for c in range(col):
    for r in range(row):
        mask = np.where(data[...,2] == cutoff[r+c])
        time_old = old[np.where(old[:,1] == cutoff[r+c])][:,3]
        ax[r,c].plot(Npoints,time_old,label="Old Cellgrid")
        for cs in cellsize:
            #time = np.empty()
            mask_cs = np.where(data[mask][...,1] == cs)
            time = data[mask][mask_cs][:,3]
            #print(Npoints,time)
            ax[r,c].plot(Npoints,time,label="Cellsize = {}".format(cs))
            ax[r,c].legend(loc='upper center', bbox_to_anchor=(1.2, 0.95),ncol=1, fancybox=True)
            ax[r,c].set_xlabel('Number of Points')
            ax[r,c].set_ylabel('Time (s)')
            ax[r,c].set_xscale('log')
        ax[r,c].set_title('Cutoff Distance = {}'.format(cutoff[r+c]))

## Variation of cellsize with time for diifferent cutoff distances


In [ ]:
data,old = np.array(data),np.array(old)
npt = Npoints[-1]
cs_arr = cellsize 
cs_arr

In [ ]:
col,row = 2,len(Npoints)//2
fig, (ax) = plt.subplots(row, col, sharex='col', sharey='row')
fig.subplots_adjust(hspace=0.7, wspace=0.5)
fig.set_size_inches(15,15)
for c in range(col):
    for r in range(row):
        npt = Npoints[r+c]
        for cut in cutoff[:-2]:
            m = data[(data[:,0]==npt) & (data[:,2]==cut)]
            time = m[:,3]
            ax[r,c].plot(cs_arr,time,label="Cutoff = {}".format(cut))
            ax[r,c].legend(loc='upper center', bbox_to_anchor=(1.2, 0.95),ncol=1, fancybox=True)
            ax[r,c].set_xlabel('Cellsize')
            ax[r,c].set_ylabel('Time (s)')
            ax[r,c].autoscale()
            #ax[r,c].set_xscale('log')
        ax[r,c].set_title('Npoints = {}'.format(npt))